In [33]:
import cv2
import numpy as np
import glob
import pickle

In [34]:
class SIFTDescriptor:
    def __init__(self, vector_size):
        self.vector_size = vector_size
        self.sift = cv2.xfeatures2d.SIFT_create()
    
    def describe(self, image_path):
        image = cv2.imread(image_path, cv2.COLOR_BGR2GRAY)
        try:
            kps = self.sift.detect(image, None)
            kps = sorted(kps, key=lambda x: -x.response)[:self.vector_size]
            kps, dsc = self.sift.compute(image, kps)
            dsc = dsc.flatten()

            needed_size = (self.vector_size * 128) # 128 is the size of a SIFT descriptor
            if dsc.size < needed_size:
                dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
        except cv2.error as e:
            print('Error: ', e)
            return None
        
        return dsc        

In [36]:
# read the image files
descriptor = SIFTDescriptor(32)
result = {}
for file in glob.glob("../detection/pos/*.jpg"):
    dsc = descriptor.describe(file)
    result[file] = dsc

f = open("features.pck", "wb")
f.write(pickle.dumps(result))
f.close()
# with open("features.pck", 'w') as fp:
#     pickle.dump(result, fp)

In [38]:
class Searcher:
    def __init__(self, index_path="features.pck"):
        self.index = pickle.loads(open(index_path, "rb").read())
    
    def search(self, query_features):
        results = {}
        
        for (k, features) in self.index.items():
            
            d = self.euclidean_distance(features, query_features)
            results[k] = d
        
        results = sorted([(v, k) for (k, v) in results.items()])
        
        return results
    
    def euclidean_distance(self, featuresA, featuresB):
        return np.linalg.norm(featuresA - featuresB)
            

In [41]:
searcher = Searcher()
query_features = descriptor.describe("watch_sample.png")
query_result = searcher.search(query_features)
print(query_result)

[(2796.952, '../detection/pos/175.jpg'), (2814.9202, '../detection/pos/935.jpg'), (2833.9495, '../detection/pos/63.jpg'), (2840.5967, '../detection/pos/920.jpg'), (2845.7847, '../detection/pos/635.jpg'), (2851.8318, '../detection/pos/425.jpg'), (2857.754, '../detection/pos/645.jpg'), (2864.4382, '../detection/pos/633.jpg'), (2865.100870824621, '../detection/pos/398.jpg'), (2865.7312, '../detection/pos/894.jpg'), (2867.2642, '../detection/pos/592.jpg'), (2869.5696, '../detection/pos/313.jpg'), (2875.604458196572, '../detection/pos/385.jpg'), (2883.0045091882876, '../detection/pos/723.jpg'), (2883.0768286675957, '../detection/pos/56.jpg'), (2886.322, '../detection/pos/696.jpg'), (2894.0579, '../detection/pos/597.jpg'), (2894.8096, '../detection/pos/37.jpg'), (2896.663, '../detection/pos/430.jpg'), (2897.351, '../detection/pos/482.jpg'), (2899.6396, '../detection/pos/541.jpg'), (2903.264541856288, '../detection/pos/294.jpg'), (2907.3325, '../detection/pos/678.jpg'), (2909.482600051081, '.